<h1><center>Test Project - Computational Finance<!center></h1>
<h2><center>Complete Market Models & Pricing Library<!center></h2>
<h3><center>Date: 2023-03-02<!center></h3>
<h3><center>Author: Alexander Nadjalin<!center></h3>

This is the solution for the test project in Computational Finance for The Pyton Quants Platinum Certificate Program, January 2023 cohort.

### Task

Write a pricing library for European plain vanilla and exotic options making use of geometric Brownian motion as the stochastic process. The pricing shall be based on Monte Carlo simulation.

The pricing library shall have:

* a class for the risk-neutral discounting (taking the short rate as parameter, generating discount factors as required, etc.)
* a class to model the parameters of the underyling risk factor and a European option (initial value, volatility, time-to-maturity, dividends, strike price, option type etc.)
* a class to value European vanilla options based on Monte Carlo simulation
* a class to value European lookback options — with either fixed or floating strike (see https://en.wikipedia.org/wiki/Lookback_option)
* a class to value European Asian options — with either fixed price or fixed strike (see https://en.wikipedia.org/wiki/Asian_option)

# Table of contents
1. [About this notebook](#aboutthisnotebook)
2. [Numerical procedure](#numericalprocedure)
3. [The underlying stock price](#theunderlyingstockprice)
    1. [The dynamics](#thedynamics)
    2. [Simulating the GBM](#simulatingthegbm)
4. [Option pricing](#optionpricing)
    1. [European vanilla option](#vanillaoption)
    2. [European lookback options](#lookbackoption)
    3. [European Asian option](#asianoption)
5. [Results](#results)

# About this notebook
## Assumptions
* We are working under the risk neutral density $Q$.
## Error handling
* There is no error handling or warnings included in this solution.
## Code
* The code in this Notebook is not optimized in any way.

# Numerical procedure
Stuff.

# The underlying stock price
Stuff.

## The dynamics
The underlying stock price follows a geometric Brownian motion (GBM)

$$
\begin{align}
dS_t = r S_t dt + \sigma S_t dZ_t
\tag{1}
\end{align}
$$

where:
* $S_0 \in R_{>0}$ is the underlying stock price at time *t*
* $r$ is the constant short rate
* $\sigma \in R_{>0}$ is the constant volatility
* $Z_t$ an arithmetic Brownian motion.

Because we are working under the risk neutral density $Q$, $\mu$ is replaced by the risk-free rate as part of the no-arbitrage assumption. The drift part of equation above is $r S_t$ and the diffusion part is $\sigma S_t$.

To represent the return of the underlying stock price $S$, we can model it by using

$$
\begin{align}
\frac{dS}{S} = r dt + \sigma dW
\tag{2}
\end{align}
$$

where $\sigma$ is the constant annualized stock price volatility and $Z$ is the GBM.

## Simulating the GBM
We can use the *Euler scheme* to discretely simulate $S(t)$. The EM is defined from (2) recursively as

$$
\begin{align}
S_t = S_{t - \Delta t} e^{\biggl(\biggl(r - \frac{\sigma^2}{2} \biggr) \Delta t + \sigma \sqrt{\Delta t} z\biggr)}
\tag{3}
\end{align}
$$

where $z$ is a standard normally distributed random variable. The value of $S$ at time $t$, is dependant on the value of $S$ at time $t - 1$, the random variable $z$, and the constant parameters $\Delta t$, $\sigma$ and $r$.

### Code for simulating the GBM ###
Below is the Python code to simulate one complete path of $S$ from $t = 0$ to $t = 1$ with $\Delta t = 1 / 365$. $\Delta t$ is chosen to represent the number of days in a standard year (no leap year).

In [ ]:
# Imports
import numpy as np
import random
import pandas as pd
import math
from scipy import stats
import matplotlib.pyplot as plt

# Notebook magic commands for plots
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Option pricing
Stuff.

## European vanilla option
Stuff.

## European lookback option
Stuff.

## European Asian option
Stuff.

# Results
Stuff.